# Chapter 2 : Write to Orion

Let's assume you have a NGSI entity ready to be sent to the Orion Context Broker.<br>
Let's continue with the tutorial of the previous chapter.

In [ ]:
# our basic NGSI entity from the previous chapter

from datetime import datetime
from pyngsi.ngsi import DataModel

m = DataModel(id="Room1", type="Room")
m.add("dateObserved", datetime.now())
m.add("pressure", 720)
m.add("temperature", 23.0)

m.pprint()

## The SinkOrion class

At this point you should have an Orion server up and running.<br>
If not you could run your own local Docker-based server, as described in the [Appendix](appendix.ipynb).

**dir()** and **help()** are always useful.

In [ ]:
# there are many other sinks mainly to help develop your agent
# use help() for info
# for the moment just focus on Orion
from pyngsi.sink import SinkOrion

# have a look at the init args
help(SinkOrion.__init__)

## Init the Sink

In [ ]:
# use our local Orion server
sink = SinkOrion()

## Write entity

In [ ]:
# by default it is a silent operation
# it's ok unless an exception is raised
sink.write(m.json())

## Update entity

In [ ]:
# update our entity : one degree more !
m.add("dateObserved", datetime.now())
m.add("temperature", 24.0)
m.pprint()

# write again
# it should update the entity in Orion
sink.write(m.json())

## Visualize result

For the moment we have no means - using pyngsi - to visualize the result of writing to Orion.<br>
It will be the point of [Chapter 5](chapter5_advanced.ipynb).<p>

In the meanwhile you can [request](https://fiware-orion.readthedocs.io/en/master/user/walkthrough_apiv2/index.html#query-entity) entities directly from the Orion API.<br>
Or as a first step just don'use Orion and replace SinkOrion with SinkStdout.

In [ ]:
from pyngsi.sink import SinkStdout

sink = SinkStdout()

sink.write(m.json)

## Constuctor args in detail

The default init args target a local installation.<br>
If needed set the **hostname** and **port** according to your Orion server.<p>

**baseurl** SHOULD not be modified.<p>

**service** and **servicepath** have to do with Fiware [multi-tenancy](https://fiware-orion.readthedocs.io/en/master/user/multitenancy/index.html).<br>
When provided, **Fiware-Service** and **Fiware-ServicePath** HTTP Headers are set accordingly.<p>

**useragent** is the **User-Agent** HTTP Header sent to Orion.<br>
It is set by **pyngsi** with its own version. You can override it.<p>

**proxy** allows you to configure a proxy between the agent and Orion, for debugging purpose.<p>

**token** allows you to provide an OAuth2 token for Orion authentication.<br>
Default is no authentication.<p>
Setting a token programmatically is NOT recommended in production mode - for security reasons.<p>
You can safely use one of the two above methods :
- set the environment variable **ORION_TOKEN**
- if using Docker, set a docker secret named **orion_token**

## Next

Go to [Chapter 3](chapter3_source.ipynb)